In [2]:
import copy
import itertools as it
import numpy as np
import yaml
import os

def expand_config(dict_config):
    keys, values = zip(*dict_config.items())
    permutations_dicts = [dict(zip(keys, v)) for v in it.product(*values)]
    return permutations_dicts

In [2]:
def generate_str_config(config):
    config_list = []
    
    lr = config['lr_base']
    epochs = config['epochs']
    output_dim = config['output_dim']
    dropout_rate = config['dropout_rate']
    lambdas = config['lambdas']
    top_anomaly_frames = config['top_anomaly_frames']
    top_normal_frames = config['top_normal_frames']
    margin = config['margin']
    optimizer = config['optimizer']
    seed = config['seed']
    
    config_list.append(f'--lr_base {lr}')
    config_list.append(f'--epochs {epochs}')
    config_list.append(f'--output_dim {output_dim}')
    config_list.append(f'--dropout_rate {dropout_rate}')
    config_list.append(f'--lambdas {lambdas}')
    config_list.append(f'--top_anomaly_frames {top_anomaly_frames}')
    config_list.append(f'--top_normal_frames {top_normal_frames}')
    config_list.append(f'--margin {margin}')
    config_list.append(f'--optimizer {optimizer}')
    
    config_list.append(f"--exps_dir 'exps/grid_search/triplet_{optimizer}_{dropout_rate}_{lambdas}_{top_anomaly_frames}_{top_normal_frames}_{margin}/{seed}/'")
    
    return config_list

def generate_bash_script(config, cuda_devices, generate_func, filename):
    n_gpus = len(cuda_devices)
    full_config = 'cd ..'
    for grid_iter, conf in enumerate(expand_config(config)):
        base_arg = f"CUDA_VISIBLE_DEVICES={cuda_devices[grid_iter%n_gpus]} python TrainingAnomalyDetector_public.py --features_path 'data/anomaly_features' --annotation_path 'Train_Annotation.txt' --network_name 'TripletAnomalyDetector' --objective_name 'triplet_objective_sampling'"

        new_task = copy.deepcopy(base_arg)
        
        res_config = generate_func(conf)
        args = ' '.join(res_config)
        new_task += ' '+args
        
        if (grid_iter+1)%n_gpus!=0: 
            new_task += ' &'
        else:
            new_task += '\nwait'
        full_config += '\n' + new_task if len(full_config) else new_task
    print(len(expand_config(config)))
    with open (f'./scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [12]:
configs = {
    'lr_base': [0.01],
    'epochs': [20],
    'output_dim': [128],
    'dropout_rate': [0.1, 0.5],
    'lambdas': [8e-5, 8e-3],
    'top_anomaly_frames': [3, 5],
    'top_normal_frames': [3, 5],
    'margin': [0.02, 0.2],
    'optimizer': ['adam', 'adadelta'],
    'seed': [42, 77, 81911]
}
    
cuda_devices = [0,1,2]*2

generate_bash_script(configs, cuda_devices, generate_str_config, 'grid_search.sh')

192


# SimCLR

In [3]:
def generate_str_config(config):
    config_list = []
    
    lr = config['lr_base']
    epochs = config['epochs']
    lambdas = config['lambdas']
    optimizer = config['optimizer']
    seed = config['seed']
    dropout_rate = config['dropout_rate']
    batch_size = config['batch_size']
    
    config_list.append(f'--lr_base {lr}')
    config_list.append(f'--epochs {epochs}')
    config_list.append(f'--dropout_rate {dropout_rate}')
    config_list.append(f'--lambdas {lambdas}')
    config_list.append(f'--optimizer {optimizer}')
    config_list.append(f'--batch_size {batch_size}')
    
    config_list.append(f"--exps_dir 'exps/SimCLR_grid_search/{optimizer}_{lr}_{batch_size}/'")
    
    return config_list

def generate_bash_script(config, cuda_devices, generate_func, filename):
    n_gpus = len(cuda_devices)
    full_config = 'cd ..'
    for grid_iter, conf in enumerate(expand_config(config)):
        base_arg = f"CUDA_VISIBLE_DEVICES={cuda_devices[grid_iter%n_gpus]} python TrainingAnomalyDetector_SimCLR.py  --features_path_1 'features1' --features_path_2 'features2' --objective_name 'SimCLRLoss' --network_name 'TripletAnomalyDetector' --annotation_path 'Train_Annotation.txt'"

        new_task = copy.deepcopy(base_arg)
        
        res_config = generate_func(conf)
        args = ' '.join(res_config)
        new_task += ' '+args
        
        if (grid_iter+1)%n_gpus!=0: 
            new_task += ' &'
        else:
            new_task += '\nwait'
        full_config += '\n' + new_task if len(full_config) else new_task
    print(len(expand_config(config)))
    with open (f'./scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [4]:
configs = {
    'lr_base': [0.01, 0.001],
    'epochs': [5],
    'batch_size': [64, 128, 256],
    'dropout_rate': [0.5],
    'lambdas': [8e-5],
    'optimizer': ['adam', 'adadelta'],
    'seed': [42]
}
    
cuda_devices = [0,1,2,3,4,5]

generate_bash_script(configs, cuda_devices, generate_str_config, 'SimCLR_grid_search.sh')

12
